In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import geopandas as gpd

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [2]:
path = '../raw_data/LB9LC30F6KALA6554.csv'
raw_data = pd.read_csv(path, low_memory=False)
print(raw_data.shape)

(536139, 89)


# 1.列处理

## 1.1 删除全为空的列

In [3]:
deal_data = raw_data.dropna(axis=1, how='all')
print(deal_data.shape)

(536139, 66)


## 1.2 删除内部值全相同的列

In [4]:
same_col = []
same_col_dict = {}

for col in deal_data.columns:
    if len(deal_data[col].unique()) == 1:
        print(col, deal_data[col].unique())
        same_col.append(col)
        same_col_dict[col] = list(deal_data[col].unique())
    else:
        pass

deal_data = deal_data.drop(same_col, axis=1)

print(deal_data.shape)
print(deal_data.columns)

VIN ['LB9LC30F6KALA6554']
(536139, 65)
Index(['消息类型', '数据采集时间', '车辆状态', '充电状态', '运行模式', '车速', '累计里程', '总电压', '总电流',
       'SOC', 'DC-DC状态', '档位', '绝缘电阻', '驱动电机个数', '驱动电机序号', '驱动电机状态',
       '驱动电机控制器温度', '驱动电机转速', '驱动电机转矩', '驱动电机温度', '电机控制器输入电压', '电机控制器直流母线电流',
       '燃料电池电压', '燃料电池电流', '燃料电池燃料消耗率', '燃料电池温度探针总数', '氢系统中最高温度',
       '氢系统中最高温度探针号', '氢气最高浓度', '氢气最高浓度传感器代号', '氢气最高压力', '氢气最高压力传感器代号',
       '高压DC-DC状态', '曲轴转速', '发动机燃料消耗率', '定位状态', '经度', '纬度', '最高电压电池子系统号',
       '最高电压电池单体代号', '电池单体电压最高值', '最低电压电池子系统号', '最低电压电池单体代号', '电池单体电压最低值',
       '最高温度子系统号', '最高温度探针号', '最高温度值', '最低温度子系统号', '最低温度探针号', '最低温度值',
       '最高报警等级', '通用报警标志', '绝缘报警', '报警标志位解析', '可充电储能装置故障总数N1', '可充电储能装置故障代码列表',
       '驱动电机故障总数N2', '发动机故障总数N3', '其他故障总数N4', '单体电池(可充电储能子系统)总数',
       '单体电池包总数(电压上传)', '单体电池电压值列表', '单体电池(可充电储能子系统)温度探针总数', '单体电池包总数(温度上传)',
       '单体电池温度值列表'],
      dtype='object')


## 1.3时间处理

In [5]:
deal_data['time'] = deal_data['数据采集时间'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
del deal_data['数据采集时间']
deal_data.sort_values(by='time', inplace=True)
deal_data.reset_index(drop=True, inplace=True)

# 2.行处理

## 2.1删除全为空的行

In [6]:
deal_data = deal_data.dropna(axis=0, how='all')
print(deal_data.shape)

(536139, 65)


## 2.2删除重复行

In [7]:
deal_data.drop_duplicates(subset=['time'], inplace=True)
deal_data.reset_index(drop=True, inplace=True)
print(deal_data.shape)

(508681, 65)


## 2.3筛选有用信息

In [8]:
for col in deal_data.columns:
    if col != 'time':
        print(col,str(deal_data[col].dtype), deal_data[col].unique())

消息类型 int64 [2 3]
车辆状态 float64 [ 1.  2. nan 12. 13. 43. 44.]
充电状态 float64 [  3.   4.   1.   2.  nan   0. 141. 139.]
运行模式 float64 [  1.  nan 255.]
车速 float64 [0.0000e+00 5.0000e+00 1.1000e+01 1.6000e+01 1.0000e+01 1.5000e+01
 9.0000e+00 6.0000e+00 3.0000e+01 4.3000e+01 4.2000e+01 3.8000e+01
 3.4000e+01 3.6000e+01 4.0000e+01 7.0000e+00 4.6000e+01 4.9000e+01
 4.8000e+01 5.1000e+01 5.2000e+01 5.3000e+01 5.0000e+01 3.3000e+01
 3.9000e+01 4.7000e+01 1.3000e+01 2.0000e+01 4.5000e+01 1.2000e+01
 3.2000e+01 4.1000e+01 4.4000e+01 2.5000e+01 3.7000e+01 2.1000e+01
 8.0000e+00 1.4000e+01 2.8000e+01 3.5000e+01 5.4000e+01 3.1000e+01
 1.0000e+00 2.3000e+01 2.4000e+01 2.7000e+01 1.8000e+01 1.9000e+01
 2.6000e+01 4.0000e+00 3.0000e+00 2.0000e+00 1.7000e+01 2.2000e+01
 2.9000e+01 5.5000e+01 5.6000e+01 5.7000e+01 5.8000e+01 6.0000e+01
 5.9000e+01 6.2000e+01 6.1000e+01        nan 6.5348e+03 6.5349e+03
 6.5350e+03 6.5346e+03 6.5353e+03 6.5345e+03 6.4000e+01 6.3000e+01
 6.5352e+03 6.5351e+03 6.5355e+03 6.5354

最后有意义的信息及异常信息
| 数据表示内容 | 描述及要求 | 备注 |
|:---:| :------|:----|
| 车辆状态 | 1：启动；2：熄火 | 出现异常数据12（106条）、13（8条）、43（1条）、44（1条）和92个nan |
| 充电状态 | 1：停车充电；2：行驶充电；<br>3：未充电；4：充电完成 | 出现异常数据0（114条）、141（1条）、139（1条）和92个nan |
| 运行模式 | 1：纯电；2：混动；3：燃油 | 出现255（0XFF）代表无效（114条），和92个nan |
| 车速 | 正常值全为整数，0-65km/h | 出现异常数据6534(116条)和92个nan |
| 累计里程 |  | 出现116条异常数据，出现92个nan |
| 总电压 | 600V左右 | 出现92个nan |
| 总电流 | -400A到400A | 出现92个nan |
| SOC | 0-100% | 出现92个nan |
| DCDC状态 | 1：工作；2：断开 | 出现异常值8（114条）、201（1条）、199（1条），和92个nan |
| 档位 | 为10进制数表示的8位2进制数 | 92个nan |
| 绝缘电阻 | 0-50000KΩ | 92个nan |
| 电池单体电压最高值 | 均值3.3V | 206个nan |
| 最高电压电池单体代号 | 电池代号1-180，出现频率差别很大 | 206个nan |
| 电池单体电压最低值 | 均值3.25V | 206个nan |
| 最低电压电池单体代号 | 电池代号1-180，出现频率差别很大 | 206个nan |
| 最高温度子系统号 |  | 206个nan |
| 最高温度值 |  |  206个nan |
| 最高温度探针号 |  | 206个nan |
| 最低温度子系统号 |  | 206个nan |
| 最低温度值 |  | 206个nan |
| 最低温度探针号 |  | 206个nan |
| 经纬度及定位状态 | 定位状态1为无效定位 | 也出现经纬度4294异常值，208个nan |
| 最高报警等级 | 故障代码0-3；0：无故障；1：1级故障，不影响车辆驾驶；2：2级故障，影响车辆性能，需驾驶员限制行驶；3：3级故障，驾驶员应立即停车或请求救援 | 208个nan |
| 通用报警标志 | 详见字段表18，由十进制数表示的多位二进制数，0：正常；2048：二进制数第11位为1，表示绝缘报警 | 2048出现1次，报警标志位解析为1次绝缘报警，'绝缘报警''可充电储能装置故障总数N1''可充电储能装置故障代码列表'互相关联。 |
| 各种报警状态 | 出现对应报警则值为1，否则为nan |  |
| 驱动电机故障总数N2 |  | 208个nan |
| 发动机故障总数N3 |  | 208个nan |
| 其他故障总数N4 | N4个其他故障 | 208个nan |
| 其他故障代码列表 | 与N4保持一致 |  |
| 单体电池(可充电储能子系统)总数、单体电池包总数(电压上传)、单体电池电压值列表 |  | 数据异常 |
| 单体电池(可充电储能子系统)温度探针总数、单体电池包总数(温度上传)、单体电池温度值列表 |  | 数据异常 |
| *驱动电机个数 | 有效值1-253，推测正常值应为1 | 出现异常值12（65条），和143个nan |


大致可以看出208条数据是有异常或是缺失值的，‘异常+缺失’对应车辆状态、充电状态、运行模式等为114/116+92；对应故障报警为0+208；对应驱动电机相关为65+143。

下面这几列参数为形式为比值的字符串，例如'1:19'，需要处理成数值型数据。且存在异常数据。


| 数据表示内容 | 描述及要求 | 备注 |
|:---:| :------|:----:|
| 驱动电机序号 | 有效值1-253，基本为1 | 对应‘驱动电机个数’为12时出现异常值<br>如‘1:0 2:0 3:160 4:215 5:215 6:217 7:216 8:216 9:214 10:216 11:214 12:214’,143个nan |
| 驱动电机状态 | 1：耗电；2：发电；3：关闭 | 对应‘驱动电机个数’为12时出现异常值<br>如‘1:255 2:8 3:12 ... 11:12 12:12’,143个nan |
| 驱动电机控制器温度 | 全为整数，单位°C | 对应‘驱动电机个数’为12时出现异常值<br>如‘1:215 2:-39 3:210 ... 11:213 12:210’,143个nan |
| 驱动电机转速 | 全为整数，单位r/min | 对应‘驱动电机个数’为12时出现异常值<br>如‘1:-1824 2:-19698 3:-16705 ... 11:-16704 12:-16705’,143个nan |
| 驱动电机转矩 | 单位Nm | 对应‘驱动电机个数’为12时出现异常值，143个nan |
| 驱动电机温度 | 全为整数，单位°C | 对应‘驱动电机个数’为12时出现异常值<br>如‘1:-33 2:34 3:-28 4:-28 ... 11:-27 12:-28’,143个nan |
| 电机控制器输入电压 | 单位V | 对应‘驱动电机个数’为12时出现异常值<br>如‘1:0.0 2:36.0 3:6452.5 4:6401.2 ... 11:1.2 12:6426.8' ,143个nan |
| 电机控制器直流母线电流 | 单位A | 对应‘驱动电机个数’为12时出现异常值<br>如‘1:-1000.0 2:-979.9 3:-998.8 4:5478.1 ... 11:5426.8 12:5401.2’,143个nan |

出现极少量（29行）燃料电池数据和（1条）发动机数据，但该车应该不是燃料电池和燃油车，这部分数据存在疑问，初步推断为错误数据


| 数据表示内容 | 描述及要求 | 备注 |
|:---:| :------|:----:|
| 燃料电池电压 燃料电池电流 燃料电池燃料消耗率 燃料电池温度探针总数 氢系统中最高温度 氢系统中最高温度探针号 氢气最高浓度 氢气最高浓度传感器代号 氢气最高压力 氢气最高压力传感器代号 高压DC-DC状态 曲轴转速 发动机燃料消耗率 |  | 异常数据 |

## 2.4 分析车辆状态等字段为nan的92条数据

In [42]:
print(deal_data[deal_data['车辆状态'].isnull()]['驱动电机个数'].value_counts())
print(deal_data[deal_data['车辆状态'].isnull()]['燃料电池电压'].value_counts())
print(deal_data.query('车辆状态 == 43.0 or 车辆状态 == 44.0 ')['燃料电池电压'].value_counts())
print(deal_data[deal_data['车辆状态'].isnull()]['曲轴转速'].value_counts())

驱动电机个数
12.0    65
Name: count, dtype: int64
燃料电池电压
307.2    27
Name: count, dtype: int64
燃料电池电压
307.2    2
Name: count, dtype: int64
曲轴转速
60929.0    1
Name: count, dtype: int64


驱动电机个数与相关字段异常的65行数据均在这92行数据中，
<br>出现燃料电池相关字段的29行数据：27行在这92行数据中，2行在车辆状态=43和44的2行数据中。
<br>出现发动机相关字段的1行数据在这92行数据中

In [45]:
print (deal_data[deal_data['燃料电池电压'].notna()]['曲轴转速'].value_counts())

曲轴转速
60929.0    1
Name: count, dtype: int64


出现发动机数据的1行数据在出现燃料电池的29行数据中。

## 2.5  分析定位状态等字段为nan的208条数据

In [10]:
y_list = ['累计里程','总电压','总电流','SOC','档位','DC-DC状态']
for i in y_list:
    print(deal_data[deal_data['定位状态'].isnull()][i].value_counts())

累计里程
1677696.7      114
338270945.4      1
338270944.7      1
Name: count, dtype: int64
总电压
0.0      114
327.4      1
327.1      1
Name: count, dtype: int64
总电流
-1000.0    114
-672.4       1
-672.9       1
Name: count, dtype: int64
SOC
0.0     114
12.0      2
Name: count, dtype: int64
档位
1.0     114
12.0      2
Name: count, dtype: int64
DC-DC状态
8.0      114
201.0      1
199.0      1
Name: count, dtype: int64


208条数据中，92条为2.4中分析的nan，另外116条数据在车速、累计里程、总电压、SOC等字段上也都出现异常，且分为114+1+1型
<br>----------------------------------------------------------------------------------------------------------------------------------------

In [20]:
y_list = ['充电状态','累计里程','总电压','总电流','SOC','档位','DC-DC状态']
for i in y_list:
    print(deal_data[deal_data['车辆状态'].isin([43,44])][i].value_counts())

充电状态
141.0    1
139.0    1
Name: count, dtype: int64
累计里程
338270945.4    1
338270944.7    1
Name: count, dtype: int64
总电压
327.4    1
327.1    1
Name: count, dtype: int64
总电流
-672.4    1
-672.9    1
Name: count, dtype: int64
SOC
12.0    2
Name: count, dtype: int64
档位
12.0    2
Name: count, dtype: int64
DC-DC状态
201.0    1
199.0    1
Name: count, dtype: int64


明显其中独立的1+1条数据为车辆状态 == 43 or 44的2行，也即充电状态 == 139 or 141的2行
<br>-----------------------------------------------------------------------------------------

注意电池单体电压值等数据nan为206条，相差2条数据，分析nan长度208与206为什么相差2条

In [21]:
print(deal_data[deal_data['定位状态'].isna() & deal_data['电池单体电压最高值'].notna()]['车辆状态'])

440498     NaN
489062    12.0
Name: 车辆状态, dtype: float64


这两条数据并非车辆状态=43和44的2行数据，如上验证

In [13]:
deal_data[deal_data['最高报警等级'].isna() & deal_data['电池单体电压最高值'].notna()]['驱动电机个数'] 

440498   NaN
489062   NaN
Name: 驱动电机个数, dtype: float64

这两条数据也并不属于驱动电机个数为异常值12的数据，如上验证

In [16]:
dif1 = deal_data[deal_data['最高报警等级'].isna() & deal_data['电池单体电压最高值'].notna()]
print(dif1['电池单体电压最高值'].value_counts())
print(dif1['电池单体电压最低值'].value_counts())
print(dif1['最高温度值'].value_counts())


电池单体电压最高值
0.360    1
0.201    1
Name: count, dtype: int64
电池单体电压最低值
40.972    1
64.780    1
Name: count, dtype: int64
最高温度值
161.0    1
214.0    1
Name: count, dtype: int64


但明显，这两行数据也是有问题的（不仅仅以上3个字段）。

## 2.6 驱动电机个数及相关字段异常分析

In [40]:
print (deal_data.query('驱动电机个数  > 1')['燃料电池电压'].value_counts())
print (deal_data.query('驱动电机个数  > 1')['曲轴转速'].value_counts())

Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)


驱动电机相关字段异常的65行数据未出现燃料电池或发动机数据